In [0]:
import numpy as np
import mysql.connector
from sqlalchemy import create_engine
import pymysql
import json
import time
from datetime import date
from dateutil.parser import parse
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup 
from tqdm import notebook
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import datetime
import re
from datetime import timedelta
import glob
import os
import math
from random import randint
from tqdm import notebook

# CNN

## coronavirus, 82 results, within 1 page

In [0]:
# keyword = coronavirus
url = 'https://www.cnn.com/search?size=100&q=corona%20virus'


In [0]:
driver = webdriver.Chrome()
driver.get(url)


In [0]:
headline_tag = driver.find_elements_by_class_name('cnn-search__result-headline')

In [0]:
xpth = "//*[@class = 'cnn-search__result-headline']/a"
links = [*map(lambda x: x.get_attribute('href'), driver.find_elements_by_xpath(xpth))]
headlines = [*map(lambda x: x.text, headline_tag)]

In [0]:
time_tag = driver.find_elements_by_class_name('cnn-search__result-publish-date')
times = [*map(lambda x: x.text, time_tag)]


In [0]:
result_body_tag = driver.find_elements_by_css_selector(r'div.cnn-search__result-body')
result_body = [*map(lambda x: x.text, result_body_tag)]

In [0]:
search_cv = pd.DataFrame(
    {'headline': headlines,
     'published_date': time,
     'link': link,
     'content': result_body
    })

In [0]:
search_cv.to_excel('CNN_coronavirus_0408.xlsx')

## Covid-19, 1800+ results

In [0]:
driver_init = webdriver.Chrome()
url_init = 'https://www.cnn.com/search?size=10&q=covid-19'
# find the place that stores how many results 

In [0]:
# keyword = coronavirus
# 1768 is the total number of results about covid-19
cnn_covid = ['https://www.cnn.com/search?size=10&q=covid-19&from='+str(x) for x in range(0, 1784, 10)]

In [0]:
cnn_covid_file = os.getcwd()+'\\cnn\\covid1'
if not os.path.exists(cnn_covid_file):
    os.makedirs(cnn_covid_file)

In [0]:
driver_cnn_covid = webdriver.Chrome()

In [0]:
# create the lists for dataframe
for idx, cnn_url in notebook.tqdm(enumerate(cnn_covid)):
    rand = randint(2,6)
    time.sleep(rand)
    # get the url, store the values of tags 
    driver_cnn_covid.get(cnn_url)
    # download the source pages
    pagesource = driver_cnn_covid.page_source
    file_name = 'covid-19' + ' ' + str(idx) + '.txt'
    name_wdir = os.path.join(cnn_covid_file, file_name)
    file = open(name_wdir, 'w',  encoding = "utf-8")
    file.write(str(pagesource))
    file.close()

In [0]:
# Get the name of all files
cnn_covid_txt_list = []
for filename in glob.glob(cnn_covid_file + '\\*.txt'):
    # Put all names in a list
    cnn_covid_txt_list.append(filename)

In [0]:
# code for selenium
#     xpth = "//*[@class = 'cnn-search__result-headline']/a"
#     links_tag = driver_cnn_covid.find_elements_by_xpath(xpth)
#     headline_tag = driver_cnn_covid.find_elements_by_class_name('cnn-search__result-headline')
#     time_tag = driver_cnn_covid.find_elements_by_class_name('cnn-search__result-publish-date')
#     result_body_tag = driver_cnn_covid.find_elements_by_css_selector(r'div.cnn-search__result-body')
#     # get values on 1 page
#     links = [*map(lambda x: x.get_attribute('href'), links_tag)]
#     headlines = [*map(lambda x: x.text, headline_tag)]
#     times = [*map(lambda x: x.text, time_tag)]
#     result_body = [*map(lambda x: x.text, result_body_tag)]
#     # store values to the large list
#     headlines_covid = headlines_covid + headlines
#     time_covid = time_covid + times
#     link_covid = link_covid + links
#     result_body_covid = result_body_covid + result_body

In [0]:
headlines_covid = []
time_covid = []
link_covid = []
result_body_covid = []

for idx, cnn_covid_txt in notebook.tqdm(enumerate(cnn_covid_txt_list)):
    name_wdir2 = os.path.join(cnn_covid_file, cnn_covid_txt)
    page_content = open(name_wdir2, "r", encoding="utf-8").read()
    soup = BeautifulSoup(page_content)
    
    # get the tags of headlines, links, published_dates, and contents
    headline_tag = soup.find_all(class_ = 'cnn-search__result-headline')
    links_tag = soup.find_all(class_ = 'cnn-search__result-headline')#['href']
    time_tag = soup.find_all(class_ = 'cnn-search__result-publish-date')
    result_body_tag = soup.find_all(class_ = 'cnn-search__result-body')
    
    # get the values of headlines, links, published_dates, and contents
    links = [idx] * 10
    result_body = [idx] * 10
    times = [idx] * 10 
    headlines = [idx] * 10
    if len([*map(lambda x: ''.join(x.text.split('\n')), headline_tag)]) != 0:
        links = [*map(lambda x: x.a['href'], links_tag)]
        headlines = [*map(lambda x: ''.join(x.text.split('\n')), headline_tag)]
        times = [*map(lambda x: ''.join(x.text.split('\n')), time_tag)]
        result_body = [*map(lambda x: ''.join(x.text.split('\n')), result_body_tag)]
    
    # store values to the list
    headlines_covid = headlines_covid + headlines
    time_covid = time_covid + times
    link_covid = link_covid + links
    result_body_covid = result_body_covid + result_body
    

In [0]:
search_covid = pd.DataFrame(
    {'headline': headlines_covid,
     'published_date': time_covid,
     'link': link_covid,
     'content': result_body_covid
    })

In [0]:
search_covid

,headline,published_date,link,content
0,Coronavirus pandemic alters life as we know it,"Apr 7, 2020",//www.cnn.com/world/live-news/coronavirus-pand...,Louisiana Gov. Joh...
1,Trump on coronavirus: promises vs. reality,"Apr 5, 2020",//www.cnn.com/videos/politics/2020/04/04/trump...,As the Covid-19 pa...
2,Revised coronavirus model predicts fewer death...,"Apr 7, 2020",//www.cnn.com/2020/04/07/health/ihme-updated-c...,An influential mod...
3,US military transferred 18 Covid-19 patients a...,"Apr 8, 2020",//www.cnn.com/2020/04/08/politics/military-tra...,A New York City ho...
4,What California is doing right in responding t...,"Apr 8, 2020",//www.cnn.com/2020/04/08/us/california-coronav...,Within weeks of th...
...,...,...,...,...
1779,What you need to know about coronavirus on Wed...,"Mar 25, 2020",//www.cnn.com/2020/03/25/world/coronavirus-new...,"""This is not a mom..."
1780,Blame game escalates between US and China over...,"Mar 23, 2020",//www.cnn.com/2020/03/24/politics/us-china-cor...,President Donald T...
1781,FDA says it is expediting the use of a blood p...,"Mar 25, 2020",//www.cnn.com/2020/03/25/health/coronavirus-bl...,The US Food and Dr...
1782,Overview of what $2 trillion stimulus bill wil...,"Mar 25, 2020",//www.cnn.com/videos/politics/2020/03/25/stimu...,CNN's Julia Chatte...


In [0]:
mask = [*map(lambda x: type(x) == int, search_covid['headline'])]
lost_pages = np.array(search_covid[mask]['headline'].unique())

In [0]:
# re-download the lost pages
cnn_covid_lost = ['https://www.cnn.com/search?size=10&q=covid-19&from='+str(x) for x in lost_pages*10]
for idx, cnn_url in notebook.tqdm(enumerate(cnn_covid_lost)):
    rand = randint(2,6)
    time.sleep(rand)
    # get the url, store the values of tags 
    driver_cnn_covid.get(cnn_url)
    # download the source pages
    pagesource = driver_cnn_covid.page_source
    file_name = 'covid-19' + ' ' + str(idx) + '.html'
    name_wdir = os.path.join(cnn_covid_file, file_name)
    file = open(name_wdir, 'w',  encoding = "utf-8")
    file.write(str(pagesource))
    file.close()

In [0]:
# Get the name of all files
cnn_covid_html_list = []
for filename in glob.glob(cnn_covid_file + '\\*.html'):
    # Put all names in a list
    cnn_covid_html_list.append(filename)

In [0]:
cnn_covid_html_list

['C:\\Users\\Willa\\Spinach\\UCD\\Hackathon\\newscrape\\cnn\\covid1\\covid-19 0.html',
 'C:\\Users\\Willa\\Spinach\\UCD\\Hackathon\\newscrape\\cnn\\covid1\\covid-19 1.html',
 'C:\\Users\\Willa\\Spinach\\UCD\\Hackathon\\newscrape\\cnn\\covid1\\covid-19 2.html',
 'C:\\Users\\Willa\\Spinach\\UCD\\Hackathon\\newscrape\\cnn\\covid1\\covid-19 3.html',
 'C:\\Users\\Willa\\Spinach\\UCD\\Hackathon\\newscrape\\cnn\\covid1\\covid-19 4.html',
 'C:\\Users\\Willa\\Spinach\\UCD\\Hackathon\\newscrape\\cnn\\covid1\\covid-19 5.html',
 'C:\\Users\\Willa\\Spinach\\UCD\\Hackathon\\newscrape\\cnn\\covid1\\covid-19 6.html']

In [0]:
headlines_covid = []
time_covid = []
link_covid = []
result_body_covid = []

for idx, cnn_covid_txt in notebook.tqdm(enumerate(cnn_covid_html_list)):
    name_wdir3 = os.path.join(cnn_covid_file, cnn_covid_txt)
    page_content = open(name_wdir2, "r", encoding="utf-8").read()
    soup = BeautifulSoup(page_content)
    
    # get the tags of headlines, links, published_dates, and contents
    headline_tag = soup.find_all(class_ = 'cnn-search__result-headline')
    links_tag = soup.find_all(class_ = 'cnn-search__result-headline')#['href']
    time_tag = soup.find_all(class_ = 'cnn-search__result-publish-date')
    result_body_tag = soup.find_all(class_ = 'cnn-search__result-body')
    
    # get the values of headlines, links, published_dates, and contents
    links = [idx] * 10
    result_body = [idx] * 10
    times = [idx] * 10 
    headlines = [idx] * 10
    if len([*map(lambda x: ''.join(x.text.split('\n')), headline_tag)]) != 0:
        links = [*map(lambda x: x.a['href'], links_tag)]
        headlines = [*map(lambda x: ''.join(x.text.split('\n')), headline_tag)]
        times = [*map(lambda x: ''.join(x.text.split('\n')), time_tag)]
        result_body = [*map(lambda x: ''.join(x.text.split('\n')), result_body_tag)]
    
    # store values to the list
    headlines_covid = headlines_covid + headlines
    time_covid = time_covid + times
    link_covid = link_covid + links
    result_body_covid = result_body_covid + result_body
    

In [0]:
search_covid_lost = pd.DataFrame(
    {'headline': headlines_covid,
     'published_date': time_covid,
     'link': link_covid,
     'content': result_body_covid
    })

In [0]:
lost_idx = search_covid.iloc[mask, :].index

In [0]:
search_covid_test = search_covid.copy()

In [0]:
for page in range(len(cnn_covid_html_list)):
    lost_idx_iter = lost_idx[page*10 : (page + 1) * 10]
    search_covid_test.iloc[lost_idx_iter,:] = search_covid_lost[page*10 : (page + 1) * 10]

In [0]:
search_covid_test
mask = [*map(lambda x: type(x) == int, search_covid_test['headline'])]
lost_pages = np.array(search_covid_test[mask]['headline'].unique())

In [0]:
search_covid_test.to_excel('CNN_covid-19_0408.xlsx')